In [2]:
!pip install requests
!pip install bs4
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib

# Importamos las librerias que vamos a necesitar Para el Web Scrapping

In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import bs4

## Importamos url base

In [3]:
baseurl ='https://statsroyale.com/es'

Conseguimos un agente de usuario que pueda enviar con cada solicitud, por lo qque de 
forma predeterminada cuando el usuario solicita si envia a una pagina el agente de 
usuario es la solicitud de Python y creo que eso puede obtener mas bloques a menudo, así
que en realidad vamos a anular y eligí este aqui desde este sitio web es común que se envíe y diga que estamos en windows y estamso usando MOZILLA 


In [4]:
headers ={'User-Agent':'Mozilla/5.0 (iPhone; CPU iPhone OS 10_0 like Mac OS X) AppleWebKit/602.4.6 (KHTML, like Gecko) Version/10.0 Mobile/14A346 Safari/E7FBAF'}

Ponemos la URL  de la web de donde comenzaremos a sacar la informacion 

In [5]:
r = requests.get('https://statsroyale.com/es/cards')

usamos beatifulsoup para obtener todo de la pagina y voy a usar el analizador XML earle en 

In [6]:
soup = BeautifulSoup(r.content, 'lxml')

Extraemos todos los link que encontramos que se relacionan directamente con la informacion que queremos extraer

In [7]:
cardlist =soup.find_all('div',class_="cards__card")

guardamos los links

In [8]:
cardlinks =[]
for item in cardlist:
    for link in item.find_all('a', href= True): 
        cardlinks.append(link['href'])

Verificamos la cantidad de link que extraemos 

In [9]:
print('Numero de links:',len(cardlinks))

Numero de links: 102


En esta parte se comienza a extraer la informaciond de cada carta 

In [10]:
lista_diccionario = []

### Lista de atributos que se extraeran:
    Nombre                             name
    Nivel                              level
    Calidad                            quality
    Objetivo                           objective
    Alcance                            scope
    Velocidad                          speed
    Radio                              radio
    Tiempo de vida                     time of life
    Puntos de vida                     Life points
    Daño                               Hurt
    Daño\s                             Damage
    Daño de área                       Area damage

In [11]:
for link in cardlinks:
    r = requests.get(link, headers = headers)
    soup = BeautifulSoup(r.content,'lxml')
    #####################################################
    #Creamos variables de aquella informacion que no tiene una clase ni etiqueta que los distinga 
    # ademas que puede o no que existan estas variable en cada uno de los links
    lifepoints = 'None'
    hurt = 'None'
    damage = 'None'
    areadamage = 'None'
    leve = soup.find('tbody')
    
    try:
        #recorremos todos los valores de etiqueta con la clase CARD_TABLEVALUES
        for item in leve.find_all('tr',class_='card__tableValues'):
            if 'Puntos de vida'== item.find('strong').text.strip():
                lifepoints = item.find('td',class_ ='card__tableValue card__isActive').text.strip()
            if 'Daño'== item.find('strong').text.strip():
                hurt = item.find('td',class_ ='card__tableValue card__isActive').text.strip()
            if 'Daño/s'== item.find('strong').text.strip():
                damage = item.find('td',class_ ='card__tableValue card__isActive').text.strip()
            if 'Daño de área' == item.find('strong').text.strip():
                areadamage = item.find('td',class_ ='card__tableValue card__isActive').text.strip()
            #print(type(item.find('strong').text.strip()))
            #print(item.find('strong').text.strip(), item.find('td',class_ ='card__tableValue card__isActive').text.strip())
            #print(item.find('td',class_ ='card__tableValue card__isActive').text)
    except:
        lifepoints = 'None'
        hurt = 'None'
        damage = 'None'
        areadamage = 'None'
####################################################################

    name =soup.find('div',class_='ui__headerMedium card__cardName').text.strip()
    try:
        level = soup.find('th', class_='card__tableHeader card__isActiveHeader').text.strip()
    except:
        level = 'None'
    try:
        quality = soup.find('span',class_='card__rare').text.strip()
    except:
        try:
            quality = soup.find('span',class_='card__legendary').text.strip()
        except:
            try:
                quality = soup.find('span',class_='card__epic').text.strip()
            except:
                try:
                    quality = soup.find('span',class_='card__common').text.strip()
                except:
                    quality = 'None'
    try:
        objective = soup.find('div', class_='card__metric card__target').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        objective = 'None'
    try:
        scope = soup.find('div',class_='card__metric card__range').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        scope = 'None'
    try:
        speed = soup.find('div',class_ ='card__metric card__movementSpeed').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        speed = 'None'
    try:
        radio = soup.find('div',class_='card__metric card__radius').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        radio = 'None'
    try:
        timeoflife = soup.find('div',class_ = 'card__metric card__durationLifetime').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        timeoflife = 'None'
#Creamos la estructura del diccionario
    CARDS ={
        'name':name,
        'level':level,
        'quality':quality,
        'objective': objective,
        'scope':scope,
        'speed':speed,
        'radio': radio,
        'timeoflife':timeoflife,
        ######################################
        'lifepoints' : lifepoints,
        'hurt' : hurt,
        'damage' : damage,
        'areadamage' : areadamage,
        'link':link
    }
    #guardamos los diccionarios en una lista
    lista_diccionario.append(CARDS)

Verificamos la cantidad de diccionarios en la lista

In [12]:
# Número de Cartas
print(len(lista_diccionario))

102


# Modelando Grafo

In [13]:
# instalando rdflib 
!pip3 install rdflib

You should consider upgrading via the 'c:\users\jose-\python\python39\python.exe -m pip install --upgrade pip' command.



In [14]:
# Nodos 
from rdflib import URIRef, BNode, Literal
# Namespace 
from rdflib import Namespace
# Crear grafos
from rdflib.namespace import RDF, RDFS
# Instaciar una clase de grafo 
from rdflib import ConjunctiveGraph
import pprint

## Clash Royale
Clash Royale es el juego de lucha de ritmo rápido en el que recolectas cartas y luchas contra otros jugadores en tiempo real. Defiende tus torres y destruye las de tu enemigo para ganar la partida.
## Cartas
#### Características:
- Nombre de la carta             
- Nivel                           
- Tipo de carta                       
- Objetivo                      
- Velocidad                                  
- Radio 
- Tiempo de vida                  
- Puntos de vida
- Daño
- Daño\s   
- Daño de área 

#### Tipo de Cartas:
1. Tropa
2. Estructura
3. Hechizo

In [15]:
caracteristicas = lista_diccionario[1].keys()
print(caracteristicas)

dict_keys(['name', 'level', 'quality', 'objective', 'scope', 'speed', 'radio', 'timeoflife', 'lifepoints', 'hurt', 'damage', 'areadamage', 'link'])


In [16]:
#import pandas as pd
#from rdfpandas.graph import to_graph
#df = pd.DataFrame(lista_diccionario, columns = caracteristicas)
#print(df.set_index('name'))
#for dic in lista_diccionario[]:
 #   print(dic)

# TEST
busqueda = 'name'
for dic in lista_diccionario[:71]:
    print(dic['name'],":",dic[busqueda])

# TEST
for dic in lista_diccionario[71:83]:
    print(dic['name'],":",dic[busqueda])

# TEST 
for dic in lista_diccionario[83:]:
    print(dic['name'],":",dic[busqueda])

# Clases Extra 
## tropa
    1. Duendes
    2. Gigantes
    3. dragones
    4. principes
    5. otros
## estructuras
    1. Chozas
    2. Torres
    3. Invocadores
## Hechizos
    1. nada.

In [17]:
n = Namespace("https://statsroyale.com/es/")

In [18]:
# Insertando las URIs de la clase: Característica
name = n.name
level = n.level
quality = n.quality
objective = n.objective
scope = n.scope
speed = n.speed
radio = n.radio
timeoflife = n.timeoflife
lifepoints = n.lifepoints
hurt = n.hurt
damage = n.damage
areadamage = n.areadamage

# tipos de Quality 
rare = n.rare
epic = n.epic
common = n.common
legendary = n.legendary

# tipos de objective

groundandair = n.groundandair
building = n.building
ground = n.ground

# tipos de scope
melee = n.melee
low = n.low
high = n.high

# tipos de speed
slow = n.slow
medium = n.medium
fast = n.fast
veryfast = n.veryfast



In [19]:
#g = to_graph(df)
g= ConjunctiveGraph()
print(g)

[a rdflib:ConjunctiveGraph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']]


In [20]:
# Creando las subclases de clase maestra: Objeto y subclases sucesoras
g.add((n.troop, RDFS.subClassOf, n.card))
g.add((n.build, RDFS.subClassOf, n.card))
g.add((n.spell, RDFS.subClassOf, n.card))

g.add([rare ,RDFS.subPropertyOf, quality ])
g.add([epic ,RDFS.subPropertyOf, quality ])
g.add([common ,RDFS.subPropertyOf, quality ])
g.add([legendary ,RDFS.subPropertyOf, quality ])


g.add([groundandair , RDFS.subPropertyOf, objective ])
g.add([building , RDFS.subPropertyOf, objective ])
g.add([ground , RDFS.subPropertyOf, objective ])

g.add([melee ,RDFS.subPropertyOf, scope ])
g.add([low ,RDFS.subPropertyOf, scope ])
g.add([high ,RDFS.subPropertyOf, scope ])

g.add([slow ,RDFS.subPropertyOf, speed ])
g.add([medium ,RDFS.subPropertyOf, speed ])
g.add([fast ,RDFS.subPropertyOf, speed ])
g.add([veryfast ,RDFS.subPropertyOf, speed ])

In [21]:
# Subpropiedades de la clase Caracteristica
#g.add([name, RDFS.subPropertyOf, n.characteristic])
#g.add([level, RDFS.subPropertyOf, n.characteristic])
g.add([quality, RDFS.subPropertyOf, n.characteristic])
g.add([objective, RDFS.subPropertyOf, n.characteristic])
g.add([scope, RDFS.subPropertyOf, n.characteristic])
g.add([speed, RDFS.subPropertyOf, n.characteristic])
g.add([radio, RDFS.subPropertyOf, n.characteristic])
g.add([timeoflife, RDFS.subPropertyOf, n.characteristic])
g.add([lifepoints, RDFS.subPropertyOf, n.characteristic])
g.add([hurt, RDFS.subPropertyOf, n.characteristic])
g.add([damage, RDFS.subPropertyOf, n.characteristic])
g.add([areadamage, RDFS.subPropertyOf, n.characteristic])

#propiedades dominios y rango
g.add([timeoflife, RDFS.domain, n.spell ])
g.add([timeoflife, RDFS.domain, n.build ])




In [22]:
# caracteristicas como etiquetas
#g.add([name, RDFS.subPropertyOf, RDFS.label])
#g.add([level, RDFS.subPropertyOf, RDFS.label])
#g.add([quality, RDFS.subPropertyOf, RDFS.label])
#g.add([objective, RDFS.subPropertyOf, RDFS.label])
#g.add([scope, RDFS.subPropertyOf, RDFS.label])
#g.add([speed, RDFS.subPropertyOf, RDFS.label])
#g.add([radio, RDFS.subPropertyOf, RDFS.label])
#g.add([timeoflife, RDFS.subPropertyOf, RDFS.label])
#g.add([lifepoints, RDFS.subPropertyOf, RDFS.label])
#g.add([hurt, RDFS.subPropertyOf, RDFS.label])
#g.add([damage, RDFS.subPropertyOf, RDFS.label])
#g.add([areadamage, RDFS.subPropertyOf, RDFS.label])

# nombres de las clases principales
g.add([n.card, name , Literal('cartas')])
g.add([n.troop, name , Literal('tropas')])
g.add([n.build, name , Literal('estructuras')])
g.add([n.spell, name , Literal('hechizos')])
g.add([n.charasteristic, name, Literal('caracteristicas')])

# nombre de los tipos
g.add([rare , name, Literal('Especial')])
g.add([epic , name, Literal('Epico')])
g.add([common , name, Literal('común') ])
g.add([legendary , name, Literal('Legendaria') ])


g.add([groundandair , name, Literal('De tierra y aire') ])
g.add([building , name, Literal('Estructura') ])
g.add([ground , name, Literal('Ground') ])

g.add([melee , name, Literal('Cuerpo a cuerpo') ])
g.add([low , name, Literal('Bajo') ])
g.add([high , name, Literal('Alto') ])

g.add([slow , name, Literal('Baja')])
g.add([medium , name, Literal('Media') ])
g.add([fast , name, Literal('Alta') ])
g.add([veryfast , name, Literal('Muy Alta') ])

# Funciones de transformación

In [23]:

def fun_quality(statement):
    if(statement == 'Especial'):
        return rare
    if(statement == 'Épica'):
        return epic
    if(statement == 'Común'):
        return common
    if(statement == 'Legendaria'):
        return legendary

In [24]:
def fun_objective(statement):
    if(statement == 'De tierra y aire'):
        return groundandair
    if(statement == 'Estructuras'):
        return building
    if(statement == 'Terrestres'):
        return ground

In [25]:
def fun_scope(statement):
    if(statement == 'Cuerpo a cuerpo'):
        return melee
    else:
        num = float(statement)
        if(num < 8.0):
            return low
        else:
            return high

In [26]:
def fun_speed(statement):
    if(statement=='Baja'):
        return slow
    if(statement=='Media'):
        return medium
    if(statement=='Alta'):
        return fast
    if(statement=='Muy alta'):
        return veryfast

# TEST
busqueda = 'speed'
i=0
for dic in lista_diccionario:
    i=i+1
    print(i)
    if(dic[busqueda]!= 'None'):
        print(dic['name'],":",fun_speed(dic[busqueda]))

In [27]:
# Instanciando Cartas Tropas

for diccionario in lista_diccionario[:71]:
    g.add((URIRef(diccionario['link']), RDF.type, n.troop))
    g.add((URIRef(diccionario['link']), name, Literal(diccionario['name'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), level, Literal(diccionario['level'])))
    if (diccionario['quality']!= 'None'):
        g.add((URIRef(diccionario['link']), quality, fun_quality(diccionario['quality'])))
    if (diccionario['objective']!= 'None'):
        g.add((URIRef(diccionario['link']), objective, fun_objective(diccionario['objective'])))
    if (diccionario['scope']!= 'None'):
        g.add((URIRef(diccionario['link']), scope, fun_scope(diccionario['scope'])))
    if (diccionario['speed']!= 'None'):
        g.add((URIRef(diccionario['link']), speed, fun_speed(diccionario['speed'])))
    if (diccionario['radio']!= 'None'):
        g.add((URIRef(diccionario['link']), radio, Literal(diccionario['radio'])))
    #if (diccionario['timeoflife']!= 'None'):
        #g.add((URIRef(diccionario['link']), timeoflife, Literal(diccionario['timeoflife'])))
    if (diccionario['lifepoints']!= 'None'):
        g.add((URIRef(diccionario['link']), lifepoints, Literal(diccionario['lifepoints'])))
    if (diccionario['hurt']!= 'None'):
        g.add((URIRef(diccionario['link']), hurt, Literal(diccionario['hurt'])))
    if (diccionario['damage']!= 'None'):
        g.add((URIRef(diccionario['link']), damage, Literal(diccionario['damage'])))
    if (diccionario['areadamage']!= 'None'):
        g.add((URIRef(diccionario['link']), areadamage, Literal(diccionario['areadamage'])))
    


In [28]:
# Instanciando Estructuras
for diccionario in lista_diccionario[71:83]:
    g.add((URIRef(diccionario['link']), RDF.type, n.build))
    g.add((URIRef(diccionario['link']), name, Literal(diccionario['name'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), level, Literal(diccionario['level'])))
    if (diccionario['quality']!= 'None'):
        g.add((URIRef(diccionario['link']), quality, fun_quality(diccionario['quality'])))
    if (diccionario['objective']!= 'None'):
        g.add((URIRef(diccionario['link']), objective, fun_objective(diccionario['objective'])))
    if (diccionario['scope']!= 'None'):
        g.add((URIRef(diccionario['link']), scope, fun_scope(diccionario['scope'])))
    if (diccionario['speed']!= 'None'):
        g.add((URIRef(diccionario['link']), speed, fun_speed(diccionario['speed'])))
    if (diccionario['radio']!= 'None'):
        g.add((URIRef(diccionario['link']), radio, Literal(diccionario['radio'])))
    if (diccionario['timeoflife']!= 'None'):
        g.add((URIRef(diccionario['link']), timeoflife, Literal(diccionario['timeoflife'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), lifepoints, Literal(diccionario['lifepoints'])))
    if (diccionario['hurt']!= 'None'):
        g.add((URIRef(diccionario['link']), hurt, Literal(diccionario['hurt'])))
    if (diccionario['damage']!= 'None'):
        g.add((URIRef(diccionario['link']), damage, Literal(diccionario['damage'])))
    if (diccionario['areadamage']!= 'None'):
        g.add((URIRef(diccionario['link']), areadamage, Literal(diccionario['areadamage'])))

In [29]:
# Instanciando Hechizos
for diccionario in lista_diccionario[83:]:
    g.add((URIRef(diccionario['link']), RDF.type, n.spell))
    g.add((URIRef(diccionario['link']), name, Literal(diccionario['name'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), level, Literal(diccionario['level'])))
    if (diccionario['quality']!= 'None'):
        g.add((URIRef(diccionario['link']), quality, fun_quality(diccionario['quality'])))
    if (diccionario['objective']!= 'None'):
        g.add((URIRef(diccionario['link']), objective, fun_objective(diccionario['objective'])))
    if (diccionario['scope']!= 'None'):
        g.add((URIRef(diccionario['link']), scope, fun_scope(diccionario['scope'])))
    if (diccionario['speed']!= 'None'):
        g.add((URIRef(diccionario['link']), speed, fun_speed(diccionario['speed'])))
    if (diccionario['radio']!= 'None'):
        g.add((URIRef(diccionario['link']), radio, Literal(diccionario['radio'])))
    if (diccionario['timeoflife']!= 'None'):
        g.add((URIRef(diccionario['link']), timeoflife, Literal(diccionario['timeoflife'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), lifepoints, Literal(diccionario['lifepoints'])))
    if (diccionario['hurt']!= 'None'):
        g.add((URIRef(diccionario['link']), hurt, Literal(diccionario['hurt'])))
    if (diccionario['damage']!= 'None'):
        g.add((URIRef(diccionario['link']), damage, Literal(diccionario['damage'])))
    if (diccionario['areadamage']!= 'None'):
        g.add((URIRef(diccionario['link']), areadamage, Literal(diccionario['areadamage'])))

texto = str(g.serialize(format="n3", encoding='utf-8').decode())
for line in texto.splitlines():
    print(line)

In [30]:
# Exportar el archivo como :
g.serialize(destination='statsroyale.rdf', format='xml')

In [31]:
#imprimir tripletas
for s, p, o in g:
    print(s, p, o)

/Arrows https://statsroyale.com/es/radio 4.0
https://statsroyale.com/es/card/Freeze https://statsroyale.com/es/radio 3
https://statsroyale.com/es/card/Skeletons http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://statsroyale.com/es/troop
https://statsroyale.com/es/card/Rage https://statsroyale.com/es/quality https://statsroyale.com/es/epic
https://statsroyale.com/es/card/Inferno+Tower https://statsroyale.com/es/timeoflife 30sec
https://statsroyale.com/es/card/Mega+Knight https://statsroyale.com/es/areadamage 222
https://statsroyale.com/es/card/Elite+Barbarians http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://statsroyale.com/es/troop
https://statsroyale.com/es/card/Mini+P.E.K.K.A https://statsroyale.com/es/level 9
https://statsroyale.com/es/card/Princess https://statsroyale.com/es/scope https://statsroyale.com/es/high
https://statsroyale.com/es/card/Goblin+Cage https://statsroyale.com/es/lifepoints 800
https://statsroyale.com/es/card/Rage https://statsroyale.com/es/level 9
ht

# TEST : Remover Tripletas
for s, p, o in g:
    g.remove((s, p, o))

# inferencias

#### Clases superiores

In [32]:
clase_base = n.troop
for s in g.transitive_objects(clase_base, RDFS.subClassOf):
    print(g.value(s, name))

tropas
cartas


##### Clases inferiores

In [ ]:
clase_base = n.card
for s in g.transitive_subjects (RDFS.subClassOf,clase_base):
    print(g.value(s, name))

#### clases de recurso

In [ ]:
recurso = URIRef(lista_diccionario[1]['link'])
for s in g.transitive_objects(recurso, RDF.type):
  print(g.value(s, name))

#### Obtener la calidad de una carta

In [ ]:
#obtener calidad
carta = URIRef(lista_diccionario[1]['link'])
for s in g.objects(carta,quality):
    print(g.value(s,quality))

#### Obtener todos los atributos de la carta

In [ ]:
def atributos(card):
    atributos = list(lista_diccionario[1].keys())
    for s in g.transitive_subjects(None,card):
        for i in range(len(atributos)):
            if(g.value(s,atributos[i])!= 'None'):
                print(atributos[i],":",g.value(s,atributos[i]))

##### Obtener los daños de cada carta

In [35]:
#Obtener los daños de cada carta
!pip install tabulate
from tabulate import tabulate
#Daños
tipos = [n.troop,n.build,n.spell]
lista=[[]]
for tipo in tipos:
    for carta in g.subjects(RDF.type,tipo):
        if g.value(carta, damage)!=None:
            lista.append([g.value(carta, name),'Daño',g.value(carta, damage)])
        elif g.value(carta, areadamage)!=None:
            lista.append([g.value(carta, name),'Daño en Área',g.value(carta, areadamage)])
        elif g.value(carta, hurt)!=None:
            lista.append([g.value(carta, name),'Daño por segundo',g.value(carta, hurt)])
print(tabulate(lista,headers=['Nombre de la carta', 'Tipo de daño','Daño']))


Nombre de la carta      Tipo de daño      Daño
----------------------  ----------------  ------------------------------------------------------------
Puercos reales          Daño              49
Lanzarrocas             Daño              95
Mini P.E.K.K.A.         Daño              351
Pandilla de duendes     Daño              90
Sabueso de lava         Daño              34
Murciélagos             Daño              51
Espíritu eléctrico      Daño por segundo  82
Bandida                 Daño              160
Arquero mágico          Daño              100
Bruja                   Daño              100
Bombardero              Daño              102
Montapuercos            Daño              165
Mago eléctrico          Daño en Área      75
Dragón infernal         Daño              75–875
Bárbaros                Daño              113
Dragones esqueleto      Daño              70
Bruja madre             Daño              100
Megaesbirro             Daño              161
Duendes con lanza       Dañ

#### Clasificar por clase tropas, estructuras y hechizos

In [36]:
#Clasificar por clase tropas, estructuras y hechizos
tipos = [n.troop,n.build,n.spell]
for tipo in tipos:
    print("...............",(g.value(tipo, name)).upper(),"...............")
    for carta in g.subjects(RDF.type,tipo):
        print(g.value(carta, name))

............... TROPAS ...............
Puercos reales
Lanzarrocas
Mini P.E.K.K.A.
Barril de esqueletos
Pandilla de duendes
Sabueso de lava
Murciélagos
Espíritu eléctrico
Bandida
Arquero mágico
Bruja
Bombardero
Montapuercos
Mago eléctrico
Dragón infernal
Bárbaros
Dragones esqueleto
Bruja madre
Megaesbirro
Duendes con lanza
Leñador
Verdugo
Pillos
Mago
Bebé dragón
Espíritus de fuego
Lanzafuegos
Minero
Guardias
Ariete de batalla
Gólem de elixir
Espíritu de hielo
Megacaballero
Gólem
Cañón con ruedas
Caballero
Arqueras
Gigante noble
Máquina voladora
Gigante eléctrico
Electrocutadores
Ejército de esqueletos
Dragón eléctrico
Esqueleto gigante
Trío de mosqueteras
Horda de esbirros
Rompemuros
Gólem de hielo
Duende gigante
Chispitas
Montacarneros
P.E.K.K.A.
Cazador
Globo bombástico
Esbirros
Reclutas reales
Esqueletos
Bárbaros de élite
Príncipe oscuro
Curandera guerrera
Mosquetera
Valquiria
Bruja nocturna
Duendes
Princesa
Gigante
Pescador
Fantasma real
Duende lanzadardos
Mago de hielo
Príncipe
...

##### Subir nivel a una carta

In [37]:
#Subir nivel a una carta
def agregarNivel(x):
    for s,d,o in g.triples((x,RDF.type,None)):
        nameX=g.value(s,name)
        nivelX=int(g.value(s,level))
        nivelX+=1
    return nivelX

In [38]:
#Prueba de subir nivel
agregarNivel(URIRef("https://statsroyale.com/es/card/Musketeer"))

10

#### Pelea entre dos cartas.

In [1]:
#funcion combate
def combat(x,lifeX,y,lifeY):
    for s,d,o in g.triples((x,RDF.type,None)):
        nameX=g.value(s,name)
        if g.value(s,damage)==None:
            damX=int(g.value(s,damage))
        elif g.value(s,areadamage)!=None:
            damX=int(g.value(s,areadamage))
        elif g.value(s,hurt)!=None:
            damX=int(g.value(s,hurt))
    for s,d,o in g.triples((y,RDF.type,None)):
        nameY=g.value(s,name)
        if g.value(s,damage)==None:
            damY=int(g.value(s,damage))
        elif g.value(s,areadamage)!=None:
            damY=int(g.value(s,areadamage))
        elif g.value(s,hurt)!=None:
            damY=int(g.value(s,hurt))
    lifeX-=damX
    lifeY-=damY
    if lifeX<=0:
        print(nameX,' vencido')
        return [0,lifeY]
    elif lifeY<=0:
        print(nameY,' vencido')
        return [lifeX,0]
    else:
        print('Ambos siguen luchando!')
        return [lifeX,lifeY]

In [42]:
#prueba de combate
musketer=URIRef("https://statsroyale.com/es/card/Musketeer")
GiantSkeleton=URIRef("https://statsroyale.com/es/card/Dark+Prince")

lifeMusketer,lifeGiantSkeleton=combat(musketer,400,GiantSkeleton,600)
print(lifeMusketer)
print(lifeGiantSkeleton)

Ambos siguen luchando!
219
394
